# 01. YOLO를 활용한 실시간 객체 탐지 (Detection & Classification)

## 01-1. YOLO 개념 및 다양한 활용 방식

### YOLO란?
- YOLO(You Only Look Once)는 단일 신경망을 사용하여 한 번의 연산으로 객체의 위치와 클래스 정보를 예측하는 객체 탐지 모델입니다.
- 기존의 R-CNN 및 Faster R-CNN과 같은 모델은 후보 영역을 먼저 찾고 이를 분류하는 방식이지만, YOLO는 전체 이미지를 한 번에 분석하여 예측을 수행하므로 속도가 빠릅니다.

### YOLO의 작동 방식
- YOLO는 CNN(Convolutional Neural Network) 기반으로 동작하며, 입력 이미지를 특징 맵으로 변환한 후 여러 개의 격자로 나누어 각 격자가 객체의 바운딩 박스와 클래스 확률을 예측합니다.
- 기존 R-CNN 기반 탐지 모델보다 속도가 빠르고, 실시간 처리에 적합합니다.

### YOLO 버전별 특징 및 차이점
- **YOLOv1~v3**: 초기 YOLO 모델로, 속도가 빠르지만 정확도가 상대적으로 낮았습니다.
- **YOLOv4**: CSPDarknet53 백본을 도입하여 정확도를 향상시킨 버전입니다.
- **YOLOv5**: PyTorch 기반으로 구현되었으며, 경량화된 모델(Tiny, Nano 등)을 포함하여 다양한 환경에서 활용할 수 있도록 최적화되었습니다.
- **YOLOv6~v8**: 최신 YOLO 버전으로, 속도와 정확도를 균형 있게 개선하며, Segmentation 및 Classification 기능도 지원합니다.

### Tiny 모델과 Nano 모델의 차이점 및 선택 이유
- **YOLO Tiny**: YOLO의 경량화된 버전으로, 연산량을 줄여 빠른 속도를 제공하지만 정확도는 낮을 수 있습니다.
- **YOLO Nano**: YOLOv5 및 v8에서 등장한 초경량 모델로, 경량화와 성능의 균형을 맞추며, Raspberry Pi와 같은 엣지 디바이스에서도 원활하게 동작할 수 있도록 최적화되었습니다.
- **Raspberry Pi 강의에서 사용할 모델**: Raspberry Pi의 연산 능력을 고려하여 YOLOv5 Nano 모델을 선택하며, 이를 TFLite로 변환하여 경량화된 형태로 실행할 것입니다.

# 02. [실습 6] YOLOv5-Nano 모델 변환 및 실행

## 02-1. Raspberry Pi에서 실행 시 필요한 설정 및 환경 구축
```bash
sudo apt update && sudo apt upgrade -y
sudo apt install python3-pip -y

# 필수 패키지 설치
pip install --upgrade pip
pip install onnx onnx2tf numpy opencv-python tensorflow torch torchvision torchaudio
```

## 02-2. YOLOv5 Nano 모델을 TFLite로 변환
### YOLOv5n 모델 다운로드
```bash
wget https://github.com/ultralytics/yolov5/releases/download/v6.0/yolov5s.pt
```
### ONNX 변환

In [ ]:
import torch

# YOLOv5 PyTorch 모델 불러오기
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5s.pt', force_reload=True)
model.eval()

# 더미 입력 데이터 (batch=1, channel=3, height=640, width=640)
dummy_input = torch.randn(1, 3, 640, 640)

# ONNX 변환
torch.onnx.export(model, dummy_input, "yolov5.onnx",
                  verbose=True, opset_version=11,
                  input_names=['input'], output_names=['output'])

print("ONNX 변환 완료: yolov5.onnx")


### TensorFlow Lite 변환

In [ ]:
import onnx
from onnx2tf import convert

# ONNX 모델 로드
onnx_model_path = "yolov5n.onnx"

# ONNX → TFLite 변환 실행
convert(
    input_onnx_file_path=onnx_model_path,
    output_folder_path="./"
)

print("TFLite 변환 완료: yolov5n.tflite")

### 모델 확인

In [ ]:
import numpy as np
import tensorflow as tf

# TFLite 모델 로드
interpreter = tf.lite.Interpreter(model_path="yolov5.tflite")
interpreter.allocate_tensors()

# 입력/출력 정보 확인
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("입력 정보:", input_details)
print("출력 정보:", output_details)

# 랜덤 입력 데이터 생성 (batch=1, channel=3, height=640, width=640)
input_data = np.random.rand(1, 3, 640, 640).astype(np.float32)

# 입력 데이터 설정 및 실행
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

# 결과 가져오기
output_data = interpreter.get_tensor(output_details[0]['index'])
print("출력 데이터:", output_data.shape)


## 02-3. 실시간 카메라 스트리밍과 연결하여 객체 탐지

### 라이브러리 및 환경 설정

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# YOLOv5 Nano TFLite 모델 로드
interpreter = tf.lite.Interpreter(model_path="yolov5n.tflite")
interpreter.allocate_tensors()

# 입력 / 출력 텐서 정보 가져오기
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# YOLOv5 Nano 모델 정보
INPUT_WIDTH = 640
INPUT_HEIGHT = 640

# 클래스 이름 (YOLOv5 기본 COCO 데이터셋 기준)
COCO_CLASSES = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck",
    "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
    "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra",
    "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee",
    "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
    "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
    "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange",
    "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch",
    "potted plant", "bed", "dining table", "toilet", "TV", "laptop", "mouse",
    "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier",
    "toothbrush"
]

### 객체 탐지 함수 정의

In [ ]:
# 객체 탐지 함수
def detect_objects(image):
    # 원본 이미지 크기 저장
    orig_height, orig_width, _ = image.shape

    # 이미지 전처리 (YOLOv5 모델 입력 크기로 조정)
    input_image = cv2.resize(image, (INPUT_WIDTH, INPUT_HEIGHT))
    input_image = np.expand_dims(input_image, axis=0).astype(np.float32) / 255.0

    # 모델 입력 설정
    interpreter.set_tensor(input_details[0]['index'], input_image)
    
    # 추론 실행
    interpreter.invoke()

    # 모델 출력 가져오기
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # 탐지된 객체 후처리
    detected_objects = []
    for detection in output_data[0]:  # YOLOv5 출력 형태: (1, 25200, 6)
        x, y, w, h, confidence, class_id = detection

        if confidence > 0.4:  # 신뢰도 기준 (0.4 이상만 표시)
            # 원본 이미지 비율에 맞게 조정
            x1 = int((x - w / 2) * orig_width)
            y1 = int((y - h / 2) * orig_height)
            x2 = int((x + w / 2) * orig_width)
            y2 = int((y + h / 2) * orig_height)

            # 클래스 ID 변환
            class_name = COCO_CLASSES[int(class_id)] if int(class_id) < len(COCO_CLASSES) else "Unknown"
            
            detected_objects.append((x1, y1, x2, y2, confidence, class_name))

    return detected_objects

### 웹캠에서 객체 탐지 실행

In [ ]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 객체 탐지 실행
    objects = detect_objects(frame)

    # 탐지된 객체를 화면에 표시
    for (x1, y1, x2, y2, confidence, class_name) in objects:
        label = f"{class_name}: {confidence:.2f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # 화면에 출력
    cv2.imshow("YOLOv5n TFLite Object Detection", frame)

    # ESC 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

# 03. YOLO를 활용한 이미지 분류(Classification)

## 03-1. YOLO를 이미지 분류용으로 활용하는 방법
- 일반적인 CNN 기반 이미지 분류 모델과 YOLO의 차이점
- YOLO의 분류 헤드 구조와 활용 가능성
- YOLOv5 및 v8에서의 Classification 적용 방식

## 03-2. YOLO 기반 분류 모델의 활용 사례
- 의료 영상에서 특정 질환 분류
- 공장 생산 라인에서 제품 불량 감지
- 일반적인 이미지 분류 태스크 적용

---

# 04. [실습 7] YOLO를 활용한 이미지 분류
## 04-1. YOLO 모델을 분류(Classification)용으로 변환
- YOLO의 Backbone을 활용하여 이미지 분류 모델로 동작하도록 설정합니다.
- Classification 전용 가중치를 사용하여 모델을 훈련합니다.
   ```bash
   python train.py --img 224 --batch 16 --epochs 50 --data classification.yaml --weights yolov5n.pt --cache
   ```

## 04-2. 사전 학습된 모델을 활용한 이미지 분류 테스트
- 사전 학습된 YOLO 모델을 활용하여 다양한 이미지에 대한 분류 테스트를 진행합니다.
   ```python
   python classify.py --weights yolov5n-cls.pt --source test.jpg
   ```
- Raspberry Pi에서 분류 모델을 실행하여 실시간으로 동작하는지 확인합니다.

## 04-3. 라즈베리파이에서 실시간 이미지 분류 구현
- OpenCV를 활용하여 카메라 입력을 실시간으로 받아 YOLO 분류 모델로 처리합니다.
- 최적화된 경량화 모델을 사용하여 Raspberry Pi에서 원활히 실행되도록 설정합니다.